# Multi-agent system with Genie on Databricks

This notebook demonstrates how to build a multi-agent system using Mosaic AI Agent Framework and [LangGraph](https://blog.langchain.dev/langgraph-multi-agent-workflows/), where [Genie](https://www.databricks.com/product/ai-bi/genie) is one of the agents.
In this notebook, you:
1. Author a multi-agent system using LangGraph.
1. Wrap the LangGraph agent with MLflow `ChatAgent` to ensure compatibility with Databricks features.
1. Manually test the multi-agent system's output.
1. Log and deploy the multi-agent system.

This example is based on the article [Use Genie in multi-agent systems](https://docs.databricks.com/aws/en/generative-ai/agent-framework/multi-agent-genie)

## Why use a Genie agent?

Multi-agent systems consist of multiple AI agents working together, each with specialized capabilities. As one of those agents, Genie allows users to interact with their structured data using natural language.

Unlike SQL functions which can only run pre-defined queries, Genie has the flexibility to create novel queries to answer user questions.

## Prerequisites

- Address all `TODO`s in this notebook.
- Create a Genie Space, see Databricks documentation ([AWS](https://docs.databricks.com/aws/genie/set-up) | [Azure](https://learn.microsoft.com/azure/databricks/genie/set-up)).

In [0]:
%pip install -U -qqq mlflow langgraph==0.3.4 databricks-langchain databricks-agents uv
dbutils.library.restartPython()


## Define the multi-agent system

Create a multi-agent system in LangGraph using a supervisor agent node directing the following agent nodes:
- **GenieAgent**: The Genie agent that queries and reasons over structured data.
- **Tool-calling agent**: An agent that calls Unity Catalog function tools.

In this example, the tool-calling agent uses the built-in Unity Catalog function `system.ai.python_exec` to execute Python code.
For examples of other tools you can add to your agents, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/agent-tool) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/agent-tool)).


#### Wrap the LangGraph agent using the `ChatAgent` interface

Databricks recommends using `ChatAgent` to ensure compatibility with Databricks AI features and to simplify authoring multi-turn conversational agents using an open source standard. 

The `LangGraphChatAgent` class implements the `ChatAgent` interface to wrap the LangGraph agent.

See MLflow's [ChatAgent documentation](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent).

#### Write agent code to file

Define the agent code in a single cell below. This lets you write the agent code to a local Python file, using the `%%writefile` magic command, for subsequent logging and deployment.


In [0]:
%%writefile agent.py
import functools
import os
from typing import Any, Generator, Literal, Optional

import mlflow
from databricks.sdk import WorkspaceClient
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
)
from databricks_langchain.genie import GenieAgent
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt import create_react_agent
from mlflow.langchain.chat_agent_langgraph import ChatAgentState
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
from pydantic import BaseModel

####################################################
## Create a GenieAgent with access to a Genie Space
####################################################

# add GENIE_SPACE_ID and a description for this space
# You can find the ID in the URL of the genie room /genie/rooms/<GENIE_SPACE_ID>
GENIE_SPACE_ID = "insert-your-genie_space_id" 
genie_agent_description = "This genie agent can answer FinOPS Questions"

#genie_agent = GenieAgent(
#    genie_space_id="01efee82a8a815e785e78a90b62e7cfc",
#    genie_agent_name="Genie",
#    description=genie_agent_description,
 #   client=WorkspaceClient(
 #       host=os.getenv("DB_MODEL_SERVING_HOST_URL"),
 #       token=os.getenv("DATABRICKS_GENIE_PAT"),
 #   ),
#)

# Initialize the agent without passing the description
genie_agent = GenieAgent(
    genie_space_id="01efee82a8a815e785e78a90b62e7cfc",
    genie_agent_name="Genie",
    client=WorkspaceClient(
        host=os.getenv("DB_MODEL_SERVING_HOST_URL"),
        token=os.getenv("DATABRICKS_GENIE_PAT"),
    ),
)


############################################
# Define your LLM endpoint and system prompt
############################################

# TODO: Replace with your model serving endpoint
# multi-agent Genie works best with claude 3.7 or gpt 4o models.
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

###################################################
## Create a SummarizeAgent using Claude 3.7 Sonnet
###################################################
class SummarizeAgent:
    def __init__(self):
        self.llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

    def invoke(self, state):
        try:
            # Find the most recent Genie message
            genie_content = None
            for msg in reversed(state["messages"]):
                if isinstance(msg, dict) and msg.get("name") == "Genie":
                    genie_content = msg.get("content", "")
                    break
                elif hasattr(msg, "name") and getattr(msg, "name") == "Genie":
                    genie_content = getattr(msg, "content", "")
                    break

            if not genie_content:
                print("SummarizeAgent: No Genie content found")
                return {
                    "messages": [
                        {
                            "role": "assistant",
                            "content": "No Genie response to summarize.",
                            "name": "SummarizeAgent",
                        }
                    ]
                }

            print(f"SummarizeAgent: Found Genie content: {genie_content[:100]}...")

            # Ask Claude to summarize the Genie response
            prompt = f"""
            Please provide a concise summary of the following information:

            {genie_content}

            Guidelines:
            1. Create a clear, bullet-point summary of the key points
            2. Highlight the most important findings or conclusions
            3. Use FinOPS terminology where appropriate and suggest Best practices
            4. Avoid repeating the original text (use your own words)
            5. Keep the summary to 3-5 bullet points
            6. Preserve any critical numerical data
            7. Use simple, direct language
            """
            # Get Claude to generate the summary
            messages = [{"role": "user", "content": prompt}]
            response = self.llm.invoke(messages)
            summary_content = response.content.strip()

            print(f"SummarizeAgent: Generated summary of length {len(summary_content)}")

            # Return the summary
            return {
                "messages": [
                    {
                        "role": "assistant",
                        "content": f"## Summary\n\n{summary_content}",
                        "name": "SummarizeAgent",
                    }
                ]
            }
        except Exception as e:
            import traceback
            print(f"SummarizeAgent error: {e}")
            print(traceback.format_exc())
            return {
                "messages": [
                    {
                        "role": "assistant",
                        "content": f"I tried to create a summary but encountered an error: {str(e)}",
                        "name": "SummarizeAgent",
                    }
                ]
            }

# Define the summarize agent
summarize_agent = SummarizeAgent()

summarize_agent_description = "The Summarize agent creates concise bullet-point summaries of the information provided by the Genie Agent."

###################################################
## Graph Agent using Claude 3.7 Sonnet
###################################################
class GraphAgent:
    def __init__(self):
        self.llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

    def invoke(self, state):
        try:
            # Find the most recent Genie message
            genie_content = None
            for msg in reversed(state["messages"]):
                if isinstance(msg, dict) and msg.get("name") == "Genie":
                    genie_content = msg.get("content", "")
                    break
                elif hasattr(msg, "name") and getattr(msg, "name") == "Genie":
                    genie_content = getattr(msg, "content", "")
                    break

            if not genie_content:
                print("GraphAgent: No Genie content found")
                return {
                    "messages": [
                        {
                            "role": "assistant",
                            "content": "No Genie response to visualize.",
                            "name": "GraphAgent",
                        }
                    ]
                }

            print(f"GraphAgent: Found Genie content: {genie_content[:100]}...")

            # Ask Claude to generate an SVG visualization based on the Genie response
            prompt = f"""
            I need you to analyze this information and create an SVG visualization:

            {genie_content}

            Based on the numerical data in this text, create a simple visualization as an SVG image.

            Guidelines:
            1. Extract the key numerical data from the text
            2. Choose an appropriate chart type (bar chart, pie chart, line graph, etc.)
            3. Return ONLY a complete, valid SVG element that can be directly embedded in Markdown
            4. The SVG should be self-contained with all styling included and can be displayed as html or markdown
            5. Include appropriate labels, title, and legend if needed
            6. Keep the SVG dimensions reasonable (around 600x400px)
            7. Use clear, readable colors and fonts

            Return ONLY the SVG code without any explanation, markdown formatting, or code blocks.
            The response should start with <svg and end with </svg>.
            """

            # Get Claude to generate the SVG
            messages = [{"role": "user", "content": prompt}]
            response = self.llm.invoke(messages)

            # Extract content based on response type
            if isinstance(response, dict):
                svg_content = response.get("content", "")
            elif hasattr(response, "content"):
                svg_content = response.content
            else:
                svg_content = str(response)

            svg_content = svg_content.strip()

            # Ensure we have valid SVG content
            if not svg_content.startswith('<svg') or not svg_content.endswith('</svg>'):
                print("GraphAgent: Invalid SVG generated, extracting SVG portion")
                # Try to extract just the SVG portion
                import re
                svg_match = re.search(r'<svg.*?</svg>', svg_content, re.DOTALL)
                if svg_match:
                    svg_content = svg_match.group(0)
                else:
                    print("GraphAgent: Could not extract SVG, using fallback")
                    # Fallback to a simple SVG if extraction fails
                    svg_content = """<svg width="600" height="200" xmlns="http://www.w3.org/2000/svg">
                        <rect width="100%" height="100%" fill="#f8f9fa"/>
                        <text x="50%" y="50%" font-family="Arial" font-size="20" text-anchor="middle">
                            Could not generate visualization from the data
                        </text>
                    </svg>"""

            print(f"GraphAgent: Generated SVG visualization of length {len(svg_content)}")

            # Create Python code that uses displayHTML to show the SVG
            display_code = f"""
                        # Display the SVG visualization
                        displayHTML('''{svg_content}''')
                        """

            # Return the code that uses displayHTML to render the SVG
            return {
                "messages": [
                    {
                        "role": "assistant",
                        "content": f"Here's a visualization based on the data:\n\n```python\n{display_code}\n```",
                        "name": "GraphAgent",
                    }
                ]
            }
        except Exception as e:
            import traceback
            print(f"GraphAgent error: {e}")
            print(traceback.format_exc())
            return {
                "messages": [
                    {
                        "role": "assistant",
                        "content": f"I tried to create a visualization but encountered an error: {str(e)}",
                        "name": "GraphAgent",
                    }
                ]
            }


# Define the graph agent
graph_agent = GraphAgent()

graph_agent_description = "The Graph agent generates graphs based on the output/answer from the Genie Agent. Try to generate grpah for each answer,if no graph is generated reply - Graph not available "


#############################
# Define the supervisor agent
#############################

# TODO update the max number of iterations between supervisor and worker nodes
# before returning to the user
MAX_ITERATIONS = 3

worker_descriptions = {
    "Genie": genie_agent_description,
    "SummarizeAgent": summarize_agent_description,
    "GraphAgent": graph_agent_description
}

formatted_descriptions = "\n".join(
    f"- {name}: {desc}" for name, desc in worker_descriptions.items()
)

system_prompt = f"""Decide between routing between the following workers or ending the conversation if an answer is provided.
{formatted_descriptions}

IMPORTANT WORKFLOW:
1. Route user questions about ITFM to the Genie agent first.
2. After Genie responds, the GraphAgent should automatically visualize the data.
3. After Genie responds, the SummarizeAgent should create a concise summary of the information.
4. End the conversation with FINISH when the user's question has been fully answered.
"""
options = ["FINISH"] + list(worker_descriptions.keys())
FINISH = {"next_node": "FINISH"}

def supervisor_agent(state):
    count = state.get("iteration_count", 0) + 1
    if count > MAX_ITERATIONS:
        return FINISH

    # Check if the last message was from Genie
    last_message = state["messages"][-1] if state["messages"] else None
    if last_message and last_message.get("name") == "Genie":
        # First route to GraphAgent
        if not any(msg.get("name") == "GraphAgent" for msg in state["messages"]):
            return {"iteration_count": count, "next_node": "GraphAgent"}
        # Then route to SummarizeAgent
        elif not any(msg.get("name") == "SummarizeAgent" for msg in state["messages"]):
            return {"iteration_count": count, "next_node": "SummarizeAgent"}

    class nextNode(BaseModel):
        next_node: Literal[tuple(options)]

    preprocessor = RunnableLambda(
        lambda state: [{"role": "system", "content": system_prompt}] + state["messages"]
    )
    supervisor_chain = preprocessor | llm.with_structured_output(nextNode)
    next_node = supervisor_chain.invoke(state).next_node

    # if routed back to the same node, exit the loop
    if state.get("next_node") == next_node:
        return FINISH
    return {
        "iteration_count": count,
        "next_node": next_node
    }


def agent_node(state, agent, name):
    result = agent.invoke(state)

    # Get the last message from the result
    last_message = result["messages"][-1]

    # Extract content based on the message type
    if isinstance(last_message, dict):
        content = last_message.get("content", "")
    else:
        content = getattr(last_message, "content", str(last_message))

    return {
        "messages": [
            {
                "role": "assistant",
                "content": content,
                "name": name,
            }
        ]
    }    

def final_answer(state):
    prompt = "Using only the content in the messages, respond to the previous user question using the answer given by the other assistant messages."
    preprocessor = RunnableLambda(
        lambda state: state["messages"] + [{"role": "user", "content": prompt}]
    )
    final_answer_chain = preprocessor | llm
    return {"messages": [final_answer_chain.invoke(state)]}


class AgentState(ChatAgentState):
    next_node: str
    iteration_count: int

#Defines AgentNodes
summarize_node = functools.partial(agent_node, agent=summarize_agent, name="SummarizeAgent")
genie_node = functools.partial(agent_node, agent=genie_agent, name="Genie")
graph_node = functools.partial(agent_node, agent=graph_agent, name="GraphAgent")

#Update Workflow Compilations
workflow = StateGraph(AgentState)
workflow.add_node("Genie", genie_node)
workflow.add_node("SummarizeAgent", summarize_node)
workflow.add_node("GraphAgent", graph_node)
workflow.add_node("supervisor", supervisor_agent)
workflow.add_node("final_answer", final_answer)

workflow.set_entry_point("supervisor")
# We want our workers to ALWAYS "report back" to the supervisor when done
for worker in worker_descriptions.keys():
    workflow.add_edge(worker, "supervisor")

# Let the supervisor decide which next node to go
workflow.add_conditional_edges(
    "supervisor",
    lambda x: x["next_node"],
    {**{k: k for k in worker_descriptions.keys()}, "FINISH": "final_answer"},
)
workflow.add_edge("final_answer", END)
multi_agent = workflow.compile()

###################################
# Wrap our multi-agent in ChatAgent
###################################
class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        request = {
            "messages": [m.model_dump_compat(exclude_none=True) for m in messages]
        }

        messages = []
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                messages.extend(
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                )
        return ChatAgentResponse(messages=messages)


    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        request = {
            "messages": [m.model_dump_compat(exclude_none=True) for m in messages]
        }
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg})
                    for msg in node_data.get("messages", [])
                )


# Create the agent object, and specify it as the agent object to use when
# loading the agent back for inference via mlflow.models.set_model()
mlflow.langchain.autolog()
AGENT = LangGraphChatAgent(multi_agent)
mlflow.models.set_model(AGENT)

## Test the agent

Interact with the agent to test its output. Since this notebook called `mlflow.langchain.autolog()` you can view the trace for each step the agent takes.

**TODO**: Replace this placeholder `input_example` with a domain-specific prompt for your agent.

## Create a Personal Access Token (PAT) as a Databricks secret
In order to access the Genie Space and its underlying resources, we need to create a PAT
- This can either be your own PAT or that of a System Principal ([AWS](https://docs.databricks.com/aws/en/dev-tools/auth/oauth-m2m) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/auth/oauth-m2m)). You will have to rotate this token yourself upon expiry.
- Add secrets-based environment variables to a model serving endpoint ([AWS](https://docs.databricks.com/aws/en/machine-learning/model-serving/store-env-variable-model-serving#add-secrets-based-environment-variables) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/store-env-variable-model-serving#add-secrets-based-environment-variables)).
- You can reference the table in the deploy docs for the right permissions level for each resource: ([AWS](https://docs.databricks.com/aws/en/generative-ai/agent-framework/deploy-agent#automatic-authentication-passthrough) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/deploy-agent#automatic-authentication-passthrough)).
  - Provision with `CAN RUN` on the Genie Space
  - Provision with `CAN USE` on the SQL Warehouse powering the Genie Space
  - Provision with `SELECT` on underlying Unity Catalog Tables 
  - Provision with `EXECUTE` on underyling Unity Catalog Functions 

In [0]:
secret_scope_name = "your-scope"
secret_key_name = "your-pat-secret-name"
secretval = dbutils.secrets.get(scope=secret_scope_name, key=secret_key_name)

In [0]:
import os
from dbruntime.databricks_repl_context import get_context

# TODO: set secret_scope_name and secret_key_name to access your PAT
secret_scope_name = "your-scope"
secret_key_name = "your-pat-secret-name"

os.environ["DB_MODEL_SERVING_HOST_URL"] = "https://" + get_context().workspaceUrl
assert os.environ["DB_MODEL_SERVING_HOST_URL"] is not None
os.environ["DATABRICKS_GENIE_PAT"] = dbutils.secrets.get(
    scope=secret_scope_name, key=secret_key_name
)
assert os.environ["DATABRICKS_GENIE_PAT"] is not None, (
    "The DATABRICKS_GENIE_PAT was not properly set to the PAT secret"
)

In [0]:
#genie_user_q=f"""Show me available Cost Models, with Amount Summary """
#genie_user_q=f"""Show me details for Actuals 2024 """
genie_user_q=f"""Summarize the cost for February 2024 """

In [0]:
from agent import AGENT

#assert genie_agent_description != "This genie agent can answer ...", (
#    "Remember to update the genie agent description for higher quality answers."
#)
input_example = {
    "messages": [
        {
            "role": "user",
            "content": genie_user_q,
        }
    ]
}
# Access 'content' using the key in the dictionary
AGENT.predict(input_example)

In [0]:
for event in AGENT.predict_stream(input_example):
  print(event, "-----------\n")

In [0]:
# Display the SVG visualization
displayHTML('''<svg width="600" height="400" xmlns="http://www.w3.org/2000/svg">
    <style>
        .bar { fill: #4285f4; }
        .bar:hover { fill: #3b75d8; }
        text { font-family: Arial, sans-serif; }
        .title { font-size: 18px; font-weight: bold; }
        .axis-label { font-size: 12px; }
        .tick { font-size: 10px; }
        .value { font-size: 11px; fill: white; font-weight: bold; }
    </style>
    
    <rect width="600" height="400" fill="#f9f9f9" />
    
    <text x="300" y="30" text-anchor="middle" class="title">Total Source Cost by Category</text>
    
    <!-- Y-axis -->
    <line x1="100" y1="50" x2="100" y2="350" stroke="#333" stroke-width="1" />
    <text x="50" y="200" text-anchor="middle" transform="rotate(-90, 50, 200)" class="axis-label">Categories</text>
    
    <!-- X-axis -->
    <line x1="100" y1="350" x2="550" y2="350" stroke="#333" stroke-width="1" />
    <text x="325" y="380" text-anchor="middle" class="axis-label">Cost (in millions $)</text>
    
    <!-- X-axis ticks -->
    <line x1="100" y1="350" x2="100" y2="355" stroke="#333" stroke-width="1" />
    <text x="100" y="365" text-anchor="middle" class="tick">0</text>
    
    <line x1="190" y1="350" x2="190" y2="355" stroke="#333" stroke-width="1" />
    <text x="190" y="365" text-anchor="middle" class="tick">2M</text>
    
    <line x1="280" y1="350" x2="280" y2="355" stroke="#333" stroke-width="1" />
    <text x="280" y="365" text-anchor="middle" class="tick">4M</text>
    
    <line x1="370" y1="350" x2="370" y2="355" stroke="#333" stroke-width="1" />
    <text x="370" y="365" text-anchor="middle" class="tick">6M</text>
    
    <line x1="460" y1="350" x2="460" y2="355" stroke="#333" stroke-width="1" />
    <text x="460" y="365" text-anchor="middle" class="tick">8M</text>
    
    <line x1="550" y1="350" x2="550" y2="355" stroke="#333" stroke-width="1" />
    <text x="550" y="365" text-anchor="middle" class="tick">10M</text>
    
    <!-- Bars -->
    <g>
        <rect x="100" y="80" width="450" height="40" class="bar" />
        <text x="105" y="105" fill="white" class="value">01. Cost Pools: $10.87M</text>
    </g>
    
    <g>
        <rect x="100" y="140" width="201" height="40" class="bar" />
        <text x="105" y="165" fill="white" class="value">02. Towers: $4.85M</text>
    </g>
    
    <g>
        <rect x="100" y="200" width="94" height="40" class="bar" />
        <text x="105" y="225" fill="white" class="value">03. Solutions: $2.26M</text>
    </g>
    
    <g>
        <rect x="100" y="260" width="94" height="40" class="bar" />
        <text x="105" y="285" fill="white" class="value">04. Business: $2.26M</text>
    </g>
</svg>''')

## Log the agent as an MLflow model

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

### Enable automatic authentication for Databricks resources
For the most common Databricks resource types, Databricks supports and recommends declaring resource dependencies for the agent upfront during logging. This enables automatic authentication passthrough when you deploy the agent. With automatic authentication passthrough, Databricks automatically provisions, rotates, and manages short-lived credentials to securely access these resource dependencies from within the agent endpoint.

To enable automatic authentication, specify the dependent Databricks resources when calling `mlflow.pyfunc.log_model().`
  - **TODO**: If your Unity Catalog tool queries a [vector search index](docs link) or leverages [external functions](docs link), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See docs ([AWS](https://docs.databricks.com/generative-ai/agent-framework/log-agent.html#specify-resources-for-automatic-authentication-passthrough) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#resources)).

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow
#from agent import GENIE_SPACE_ID, LLM_ENDPOINT_NAME, tools
from agent import GENIE_SPACE_ID, LLM_ENDPOINT_NAME
from databricks_langchain import UnityCatalogTool, VectorSearchRetrieverTool
from mlflow.models.resources import (
    DatabricksFunction,
    DatabricksGenieSpace,
    DatabricksServingEndpoint,
)
from pkg_resources import get_distribution

# TODO: Manually include underlying resources if needed. See the TODO in the markdown above for more information.
resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
    DatabricksGenieSpace(genie_space_id=GENIE_SPACE_ID),
]
#for tool in tools:
#    if isinstance(tool, VectorSearchRetrieverTool):
#        resources.extend(tool.resources)
#    elif isinstance(tool, UnityCatalogTool):
#        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="agent.py",
        input_example=input_example,
        extra_pip_requirements=[f"databricks-connect=={get_distribution('databricks-connect').version}"],
        resources=resources,
    )

## Pre-deployment agent validation
Before registering and deploying the agent, perform pre-deployment checks using the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See Databricks documentation ([AWS](https://docs.databricks.com/en/machine-learning/model-serving/model-serving-debug.html#validate-inputs) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/model-serving-debug#before-model-deployment-validation-checks))."

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data=input_example,
    env_manager="local",
)

## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "magicorange_qa"
schema = "insights"
model_name = "test_genie_model"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

## Deploy the agent

In [0]:
from databricks import agents

agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    tags={"endpointSource": "docs"},
    environment_vars={
        "DATABRICKS_GENIE_PAT": f"{{{{secrets/{secret_scope_name}/{secret_key_name}}}}}"
    },
)

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See Databricks documentation ([AWS](https://docs.databricks.com/en/generative-ai/deploy-agent.html) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/deploy-agent)).